In [1]:
import numpy as np
import pandas as pd
import requests
import re
import time
import sys
import os

In [2]:
df = pd.read_pickle('../ingestion/data/dune_2021_data.pkl')
df.head()

,rating,review,displayName,displayImageUrl,isVerified,isSuperReviewer,hasSpoilers,hasProfanity,score,timeFromCreation,user.displayName,user.accountLink,user.realm,user.userId
0,5.0,Excellent! One of the best movies I have seen ...,Kellie G,https://graph.facebook.com/v3.3/22821587819351...,False,False,False,False,5.0,6h ago,Kellie G,/user/id/979452817,RT,979452817
1,4.5,The only draw back is that its 2 hours 35 minutes,George M,https://graph.facebook.com/v3.3/1755748644/pic...,False,False,False,False,4.5,13h ago,George M,/user/id/895190686,RT,895190686
2,5.0,Fantastic. Can't wait for part 2.,Julian C,https://graph.facebook.com/v3.3/10158297875246...,False,False,False,False,5.0,1d ago,Julian C,/user/id/979451578,RT,979451578
3,2.0,Another Sci-Fi alternate universe that fails t...,Diane L,NaN,False,False,False,False,2.0,2d ago,Diane L,/user/id/979449083,RT,979449083
4,2.0,Dune 2021 (Part 1) Review - I'm a Dune fanatic...,Martin D,NaN,False,False,False,False,2.0,2d ago,Martin D,/user/id/979451296,RT,979451296


In [3]:
def generate_sample_dfs(dataframe):
    '''
    
    '''
    num_random_reviews = len(dataframe)//3
    samp_range = len(dataframe) //num_random_reviews
    df_shuff = dataframe.reindex(np.random.permutation(dataframe.index))

    df_dict = {}
    for i in range(samp_range): 
        df_dict[i+1] = df_shuff[i*num_random_reviews:(i+1)*num_random_reviews]

    return df_dict

generate_sample_dfs(df)

{1:      rating                                             review displayName  \
 879     4.0  It's good so far, looking forward to see what'...  Federico N   
 88      1.5  Haven't watched a movie as boring as this one ...    Josipa H   
 129     3.5                    Well done and true to the book.     Susan M   
 153     3.0  Beautiful. \nBut.\nI was predisposed to love t...     Susan S   
 778     5.0  Probably the best sci-fi the year. Very loyal ...     Felix A   
 ..      ...                                                ...         ...   
 241     3.0  I wish I could give this movie a better review...  Omar Ajami   
 630     5.0  Really enjoyed it, looking forward to the sequ...  Theodore N   
 595     5.0  Actor selection:  excellent.  Acting:  excelle...     James T   
 394     4.0  A great movie, that just falls short of brilli...    Ismail K   
 915     3.0  The imax viuals and sound were all that made i...      Wade Z   
 
                                        display

In [4]:
t = generate_sample_dfs(df)
t[1]

,rating,review,displayName,displayImageUrl,isVerified,isSuperReviewer,hasSpoilers,hasProfanity,score,timeFromCreation,user.displayName,user.accountLink,user.realm,user.userId
60,3.0,Effects were good but with a very dry storyline,Cedric D,https://graph.facebook.com/v3.3/10348486889774...,False,False,False,False,3.0,"Jan 11, 2022",Cedric D,/user/id/979438370,RT,979438370
9,5.0,So long since I read the books (and I read all...,Bob S,NaN,False,False,False,False,5.0,2d ago,Bob S,/user/id/977035763,RT,977035763
20,4.0,"It is a great movie with incredible direction,...",Vrettos S,https://graph.facebook.com/v3.3/14325693468688...,False,False,False,False,4.0,5d ago,Vrettos S,/user/id/977007303,RT,977007303
661,5.0,Loved this movie. It has such a sure sense of ...,Paul S,NaN,False,False,False,False,5.0,"Dec 04, 2021",Paul S,/user/id/971924257,RT,971924257
682,5.0,Amazing visuals and stuck to the story quite w...,S D,https://graph.facebook.com/v3.3/1031208955/pic...,False,False,False,False,5.0,"Dec 03, 2021",S D,/user/id/877788137,RT,877788137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,5.0,Sand. Worms. Thigh pads. Still suits. Nose tub...,Richard S,NaN,True,False,False,False,5.0,"Nov 27, 2021",Richard S,NaN,Fandango,44a2b8b4-c542-401b-8220-8d673d2cd023
604,1.5,what i hated the most about this movie is it's...,Scott M,NaN,False,False,False,False,1.5,"Dec 06, 2021",Scott M,/user/id/979372881,RT,979372881
905,3.0,"Great video effect, ,,,but,,,",M,NaN,True,False,False,False,3.0,"Nov 26, 2021",M,NaN,Fandango,6bc6babb-1ef1-4e0f-871b-d8c42adde484
326,5.0,"Love it! It's a slow build but so worth while,...",Heath T,https://graph.facebook.com/v3.3/10154034117735...,False,False,False,False,5.0,"Dec 22, 2021",Heath T,/user/id/979368463,RT,979368463


In [5]:
df_shuff = df.reindex(np.random.permutation(df.index))
df_shuff

,rating,review,displayName,displayImageUrl,isVerified,isSuperReviewer,hasSpoilers,hasProfanity,score,timeFromCreation,user.displayName,user.accountLink,user.realm,user.userId
909,5.0,It's just a beautifully shot movie. It feels l...,The G,NaN,False,False,False,False,5.0,"Nov 26, 2021",The G,/user/id/979224209,RT,979224209
820,4.0,smoother than the original with a better expla...,Rick,NaN,False,False,False,False,4.0,"Nov 28, 2021",Rick,NaN,Fandango,CDD30017-F5C1-40EB-8B1F-9D6D93A2859C
584,3.5,"The movie was enjoyable. Coloful, eye catching...",Carol Ann,NaN,False,False,False,False,3.5,"Dec 07, 2021",Carol Ann,NaN,Fandango,7b9ab9bc-1125-4356-bdcd-6edb1e6c457d
363,4.5,Really awesome! Awaiting for a second part of ...,Dariusz B,https://graph.facebook.com/v3.3/46302986570039...,False,False,False,False,4.5,"Dec 19, 2021",Dariusz B,/user/id/979242798,RT,979242798
31,5.0,"This transposition is awesome, you can really ...",Matteo P,https://graph.facebook.com/v3.3/30890689713500...,False,False,False,False,5.0,"Jan 16, 2022",Matteo P,/user/id/979325244,RT,979325244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,3.0,Disappointed. The film ends at the highest poi...,Tom O,NaN,False,False,False,False,3.0,"Dec 12, 2021",Tom O,/user/id/979378655,RT,979378655
289,5.0,Loved this movie. A thousand times better than...,Denise,NaN,True,False,False,False,5.0,"Dec 23, 2021",Denise,NaN,Fandango,7ba355c5-22c2-4be3-a6b3-e14ee07c4e3b
911,5.0,excellent movie even if you do not know anythi...,NaN,NaN,False,False,False,False,5.0,"Nov 26, 2021",NaN,NaN,Flixster,c3ed4bf8-21c4-49c3-9f75-074ae618d898
177,3.5,"A well-made, modern take on Dune that is true ...",Leon P,NaN,False,False,False,False,3.5,"Dec 30, 2021",Leon P,/user/id/979422081,RT,979422081


In [6]:
#subsetting for verified data because.. they're verified and makes things a bit simpler for computing and memory costs
verified_df = df[df['isVerified'] == True]
print(verified_df.shape)
verified_df.head()

(196, 14)


,rating,review,displayName,displayImageUrl,isVerified,isSuperReviewer,hasSpoilers,hasProfanity,score,timeFromCreation,user.displayName,user.accountLink,user.realm,user.userId
14,3.5,"Solid movie, but some parts were a little conf...",Yousef H,NaN,True,False,False,False,3.5,4d ago,Yousef H,NaN,Fandango,AADE9D8E-25B6-4BF1-AB14-25C77B35F07D
23,5.0,"not what i was expecting, but fantastic story,...",salsa,NaN,True,False,False,False,5.0,5d ago,salsa,NaN,Fandango,0af117de-2de3-4415-ab22-d6ef2ef0db4d
25,2.0,Trades off dialog and story line for loud musi...,Stoner,NaN,True,False,False,False,2.0,6d ago,Stoner,NaN,Fandango,dbdce309-d58b-4d56-9b08-6c1e4feae9e0
40,1.5,unnecessarily long and boring storyline.,Chad H,NaN,True,False,False,False,1.5,"Jan 15, 2022",Chad H,NaN,Fandango,C3936DB7-FE19-4C0A-976B-69226018A4A2
62,2.0,Don’t understand the hype. Slow and confusing.,Robin C,NaN,True,False,False,False,2.0,"Jan 10, 2022",Robin C,NaN,Fandango,A4CC7834-0B17-41D8-8E57-651678F10120


In [7]:
# combining all reviews together
# FYI, there is no need to preprocess / clean the text with huggingface pipelines:
# https://blog.paperspace.com/natural-language-processing-with-huggingface/#:~:text=The%20pipeline%20contains%20the%20pre,to%20perform%20any%20text%20preprocessing
reviews = '. '.join([i for i in verified_df.review.values])
reviews[:1000]

"Solid movie, but some parts were a little confusing where the story expected or assumed you would know ahead of time.. not what i was expecting, but fantastic story, huge scale, spectacular cgi, just stellar performance by all the actors. ready for the next !. Trades off dialog and story line for loud music. If I hadn't read the book or seen the previous Dune movie, I wouldn't have had any idea what the story was about. Even inserts one bit of locker room humor in something that is the darkest story I have ever read, really didn't fit at all.. unnecessarily long and boring storyline.. Don’t understand the hype. Slow and confusing.. Soundtrack is amazing. Visually sets a world apart from other sci fi franchises. Acting was good. Timothy and Rebecca especially.. It’s good but it’s confusing. You need to pay attention if not then you’ll get so confused. The start of the movie was great. Amazing visuals accompanied with beautiful music and plenty of action. However the movie slows down qu

In [8]:
from transformers import pipeline

In [9]:
summarizer = pipeline("summarization", 
                      model="t5-base", 
                      tokenizer="t5-base", 
                       device = 0, # comment out line if there is no CUDA enabled GPU
)
summarizer.device

device(type='cuda', index=0)

In [10]:
from nltk import sent_tokenize
import textwrap

sentences = sent_tokenize(reviews)
textwrap.wrap(reviews, 512)[0]

"Solid movie, but some parts were a little confusing where the story expected or assumed you would know ahead of time.. not what i was expecting, but fantastic story, huge scale, spectacular cgi, just stellar performance by all the actors. ready for the next !. Trades off dialog and story line for loud music. If I hadn't read the book or seen the previous Dune movie, I wouldn't have had any idea what the story was about. Even inserts one bit of locker room humor in something that is the darkest story I have"

In [11]:
char_limit = 1500
input_text = textwrap.wrap(reviews, char_limit)[0] # gpu taps out around this length string 
print(f'Input text len: {len(input_text)} \n ')
summary_text = summarizer(input_text,
                          max_length=100,
                          min_length=5,
                          do_sample=False)[0]['summary_text']
print(summary_text)

Input text len: 1496 
 
the movie slows down quite a bit after the halfway mark, and lost a lot of suspense . there is no finer rendition of this wonderful story, and heck, it even makes a decent film this time .


In [12]:
#for generating multiple randomly sampled reviews from dataset

In [13]:
dict_data = generate_sample_dfs(df)
for i in range(1, 4):
    df_sample = dict_data[i]
    reviews = '. '.join([i for i in df_sample.review.values])
    input_text = textwrap.wrap(reviews, char_limit)[0] # gpu taps out around this length string 
    print(f'Input text len: {len(input_text)} \n ')
    summary_text = summarizer(input_text,
                              max_length=200,
                              min_length=5,
                              do_sample=False)[0]['summary_text']
    print(summary_text)

Input text len: 1498 
 
the casting of Timothee Chamalet as Paul was a brilliant choice . the story is also told in a very decent way . i can not think of a better representation of the book .
Input text len: 1496 
 
i loved the 1984 movie - thought it was quite good . but this one is absolutely stunning . i can't wait for the next one, and will certainly see this again .
Input text len: 1495 
 
dune movie is an in theater third row recline chair movie . get a hazmat suit if that makes you feel more comfortable . some minor pacing issues an odd story structure by its self .
